In [17]:
import pandas as pd
import utils as ut 
from haversine import haversine

In [7]:
business = pd.read_json('./data/yelp_academic_dataset_business.json', lines=True)
checkin = pd.read_json('./data/yelp_academic_dataset_checkin.json', lines=True)

### Q1. 2019년 온타리오 주(state = ON)에서 별점이 3.0 이상인 레스토랑 중 체크인이 가장 많은 top 10의 레스토랑명을 내림차순으로 나열하세요.

### Answer
온타리오 주에서 별점이 3.0 이상인 레스토랑은 없다.

레스토랑이 아닌 Bernie Hipos Wedding Films라는 Event Planning & Service 가게가 있으나, 해당 가게의 체크인 결과가 없다.

In [10]:
state_on = business.query("state == 'ON' and stars >= 3")[['business_id', 'name', 'state', 'stars', 'categories']]
state_on

,business_id,name,state,stars,categories
52167,9fJ7gK8SaNvoe0AP_JXZiw,Bernie Hipos Wedding Films,ON,5.0,"Event Planning & Services, Videographers, Prof..."


In [9]:
state_on.merge(checkin, how='left')

,business_id,name,state,stars,categories,date
0,9fJ7gK8SaNvoe0AP_JXZiw,Bernie Hipos Wedding Films,ON,5.0,"Event Planning & Services, Videographers, Prof...",NaN


### Q1. 토론토에서 피자 가게를 창업할 계획입니다. 다만, 24시간 영업을 해야할지 고민입니다. 
### 주어진 데이터로 24시간 영업의 타당성을 검증해 보세요. (가설을 증명하는데 필요한 정보나 상황은 자유롭게 가정하세요)

1. 토론토 현황
    - 토론토 데이터는 1개 뿐이며 해당 데이터의 카테고리가 음식과 무관하므로 활용하기 어렵다. 
    - 토론토와 가장 가까운 지역의 데이터를 확인할 필요가 있다.

In [11]:
business.query("city == 'Toronto'")

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
52167,9fJ7gK8SaNvoe0AP_JXZiw,Bernie Hipos Wedding Films,,Toronto,ON,M5C 1T4,49.226695,-122.893603,5.0,7,1,None,"Event Planning & Services, Videographers, Prof...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."


In [12]:
business.query("city == 'Toronto'")['categories'].tolist()

['Event Planning & Services, Videographers, Professional Services, Video/Film Production']

과연 저곳은 정말 토론토일까 ? 

In [42]:
business.query("city == 'Toronto'")[['latitude', 'longitude']]

,latitude,longitude
52167,49.226695,-122.893603


![nn](image/image1.png)

In [13]:
toronto_lat = business.query("city == 'Toronto'")['latitude'].tolist()[0]
toronto_long = business.query("city == 'Toronto'")['longitude'].tolist()[0]

toronto = (toronto_lat, toronto_long)

In [14]:
l = list()
for x in range(len(business)): 
    lat = business.iloc[x, :]['latitude']
    long = business.iloc[x, :]['longitude']
    
    site = (lat, long)
    
    dist = haversine(toronto, site)
    l.append(dist)

In [15]:
business['dist_by_toronto'] = l

In [26]:
a = business.groupby(['state', 'city'], as_index=False).agg(
    count = ('name', 'count'),
    lat = ('latitude', 'mean'),
    long = ('longitude', 'mean')
)


In [32]:
x = (43.6608104195043, -79.36970321230831)

In [35]:
lat_list = a['lat'].tolist()
long_list = a['long'].tolist()

In [37]:
l = list()
for idx, lat in enumerate(lat_list):
    long = long_list[idx]
    d = haversine(x, (lat, long))
    l.append(d)

In [39]:
a['dist'] = l

In [41]:
a.sort_values('dist')

,state,city,count,lat,long,dist
674,OH,Sunbury,55,40.252194,-82.889044,477.755311
607,OH,Berkshire,2,40.267724,-82.924730,478.171293
608,OH,Berkshire Township,2,40.260521,-82.925710,478.862954
650,OH,Johnstown,18,40.142272,-82.712272,479.070116
636,OH,Galena,15,40.211070,-82.905456,482.266699
...,...,...,...,...,...,...
711,OR,Hillsbor,1,45.534892,-122.863234,3407.627212
708,OR,HILLSBORO,1,45.539960,-122.868184,3407.883305
750,OR,Sherwood,5,45.395781,-122.823625,3407.948088
712,OR,Hillsboro,323,45.536856,-122.877859,3408.693193
